## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2021
month = 9
program = "prepilot"
study_type = "program"
mode_of_interest = "Pilot ebike"

In [ ]:
from collections import defaultdict
import datetime
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from plots import *
import scaffolding
from uuid import UUID

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    raise Exception("The plots in this notebook are only relevant to programs")

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r df_ei
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_fuel = defaultdict(lambda: 'Other',dic_fuel)

## Collect Data From Database

In [ ]:
tq = scaffolding.get_time_query(year, month)
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)
expanded_ct = scaffolding.expand_userinputs(labeled_ct)
expanded_ct = scaffolding.data_quality_check(expanded_ct)

In [ ]:
# Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

# Mapping fuel
expanded_ct['Mode_confirm_fuel']= expanded_ct['Mode_confirm'].map(dic_fuel)
expanded_ct['Replaced_mode_fuel']= expanded_ct['Replaced_mode'].map(dic_fuel)

# Change meters to miles
scaffolding.unit_conversions(expanded_ct)

# Calculate energy impact
expanded_ct = scaffolding.energy_intensity(expanded_ct, df_ei, 'distance_miles', 'Replaced_mode', 'Mode_confirm')
expanded_ct = scaffolding.energy_impact_kWH(expanded_ct, 'distance_miles', 'Replaced_mode', 'Mode_confirm')
expanded_ct = scaffolding.CO2_impact_lb(expanded_ct, 'distance_miles', 'Replaced_mode', 'Mode_confirm')

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, expanded_ct)

## Metrics for Specific Mode

In [ ]:
data_eb = expanded_ct.query(f"Mode_confirm == '{mode_of_interest}'")

In [ ]:
quality_text = scaffolding.get_quality_text(expanded_ct, data_eb)

### Distribution of Replaced_mode attribute

In [ ]:
# Note: The portion of pilot ebikes in the pie chart below is only when the mode_confirm is different that ebike. Example: mode_confirm was Walk, replaced_mode ebike.
# mode_confrim was Car, drove alone, replaced_mode ebike.
labels_rm = expanded_ct['Replaced_mode'].value_counts(dropna=True).keys().tolist()
values_rm = expanded_ct['Replaced_mode'].value_counts(dropna=True).tolist()
plot_title="Number of trips for each replaced mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_replaced_mode%s.png' % file_suffix
pie_chart_mode(plot_title,labels_rm,values_rm,file_name)
alt_text = scaffolding.store_alt_text_pie(pd.DataFrame(values_rm, labels_rm), file_name, plot_title, './alt_text_lookup.json')

### Trips by purpose for specified mode

In [ ]:
labels_tp = data_eb['Trip_purpose'].value_counts(dropna=True).keys().tolist()
values_tp = data_eb['Trip_purpose'].value_counts(dropna=True).tolist()
plot_title=f"Number of trips for each purpose for {mode_of_interest} only\n{quality_text}"
file_name= 'ntrips_ebike_purpose%s.png' % file_suffix
pie_chart_purpose(plot_title,labels_tp,values_tp,file_name)
alt_text = scaffolding.store_alt_text_pie(pd.DataFrame(values_tp, labels_tp), file_name, plot_title, './alt_text_lookup.json')

In [ ]:
labels_eb = data_eb.Replaced_mode.value_counts(dropna=True).keys().tolist()
values_eb = data_eb.Replaced_mode.value_counts(dropna=True).tolist()
plot_title=f"Number of trips for each replaced transport mode for {mode_of_interest} only\n{quality_text}"
file_name ='ntrips_ebike_replaced_mode%s.png' % file_suffix
pie_chart_mode(plot_title,labels_eb,values_eb,file_name)
alt_text = scaffolding.store_alt_text_pie(pd.DataFrame(values_eb, labels_eb), file_name, plot_title, './alt_text_lookup.json')

### Miles for each mode replaced by the specified mode

In [ ]:
dg=data_eb.groupby('Replaced_mode').agg({'distance_miles': ['sum', 'count' , 'mean']},)
dg.columns = ['Total (miles)', 'Count' ,'Average (miles)']
dg = dg.reset_index()
dg = dg.sort_values(by=['Total (miles)'], ascending=False)

dg_dict = dict(zip(dg['Replaced_mode'], dg['Total (miles)']))
labels_m = []
values_m = []

for x, y in dg_dict.items():
    labels_m.append(x)
    values_m.append(y)

plot_title=f"Distribution of Miles Replaced by {mode_of_interest} \n{quality_text}"
file_name ='miles_ebike_replaced_mode%s.png' % file_suffix
pie_chart_mode(plot_title,labels_m,values_m,file_name)
alt_text = scaffolding.store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title, './alt_text_lookup.json')
print(dg)

### Average miles per trip for specified mode

In [ ]:
data = dg.drop((dg.query("Count < 3").index)).sort_values(by=['Average (miles)'], ascending=False)
x='Replaced_mode'
y='Average (miles)'
y2 = "Count"

plot_title=" Average Miles for each replaced mode with > 3 entries\n'Other' represents trips with a non-standard or missing replacement\n%s" % quality_text
file_name ='average_miles_replaced_mode%s.png' % file_suffix
barplot_mode(data,x,y,plot_title,file_name)
alt_text = scaffolding.store_alt_text_bar(pd.DataFrame(data['Average (miles)'].values, data.Replaced_mode), file_name, plot_title, './alt_text_lookup.json')

In [ ]:
data = dg.drop((dg.query("Count < 3").index)).sort_values(by=['Average (miles)'], ascending=False)
x='Replaced_mode'
y='Average (miles)'
y2 = "Count"

plot_title=" Average Miles for each replaced mode with > 3 entries\n%s" % quality_text
file_name ='average_miles_replaced_mode2%s.png' % file_suffix

barplot_mode2(data,x,y,y2,plot_title,file_name)
alt_text = scaffolding.store_alt_text_bar(pd.DataFrame(data['Average (miles)'].values, data.Replaced_mode), file_name, plot_title, './alt_text_lookup.json')

### Number of trips by day for specified mode

In [ ]:
fq_days = data_eb.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
fq_days = fq_days.reset_index()
fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

data = fq_days
x = 'Day of the Month'
y = 'Number of Trips'

plot_title=f"Number of {mode_of_interest} trips by day\n%s" % quality_text
file_name ='ntrips_ebike_per_day%s.png' % file_suffix

barplot_day(data,x,y,plot_title,file_name)
alt_text = scaffolding.store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month'].values), file_name, plot_title, './alt_text_lookup.json')

### Number of trips by day of week¶

In [ ]:
fq_weekdays = data_eb.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
fq_weekdays = fq_weekdays.reset_index()
fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

data = fq_weekdays
x = 'Weekday'
y = 'Number of Trips'

plot_title="Number of trips by weekday\n%s" % quality_text
file_name ='ntrips_per_weekday%s.png' % file_suffix

barplot_day(data,x,y,plot_title,file_name)
alt_text = scaffolding.store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday'].values), file_name, plot_title, './alt_text_lookup.json')